In [1]:
import pandas as pd

In [36]:
fact_returns = pd.read_csv('factor_return.csv')

In [50]:
returns_21d_raw = fact_returns.filter(regex='^fr_21d_next_raw').copy()
returns_21d_raw['time'] = fact_returns['time']
factors = returns_21d_raw[returns_21d_raw['time']>= '2015-01-01'].reset_index(drop=True)
factors['time'] = pd.to_datetime(factors['time'])

In [68]:
factors.head()

,fr_21d_next_raw_SIZE,fr_21d_next_raw_SIZENL,fr_21d_next_raw_LIQUIDITY,fr_21d_next_raw_BETA,fr_21d_next_raw_RESVOL,fr_21d_next_raw_MOMENTUM,fr_21d_next_raw_LEVERAGE,fr_21d_next_raw_VALUE,fr_21d_next_raw_EARNYLD,fr_21d_next_raw_GROWTH,fr_21d_next_raw_LTREV,fr_21d_next_raw_STREV,fr_21d_next_raw_price,time
0,-0.029944,0.010329,-0.014406,0.003392,0.043820,-0.015595,-0.013227,-0.012587,0.011419,0.002194,-0.015348,0.033077,0.000531,2015-01-05 10:00:00
1,-0.035994,0.012705,-0.016470,0.006909,0.054080,-0.015000,-0.007259,-0.014086,0.013329,0.002739,-0.017733,0.029639,0.000470,2015-01-06 10:00:00
2,-0.032634,0.009117,-0.017115,0.006934,0.054754,-0.017596,-0.002960,-0.010251,0.013611,0.004117,-0.016178,0.026539,0.000448,2015-01-07 10:00:00
3,-0.031277,0.009016,-0.015949,0.009558,0.045922,-0.016463,-0.008067,-0.010321,0.016630,0.004974,-0.017005,0.030534,0.000425,2015-01-08 10:00:00
4,-0.027636,0.006476,-0.014973,0.012897,0.034794,-0.012055,-0.009979,-0.011245,0.011686,0.006133,-0.017181,0.028227,0.000433,2015-01-09 10:00:00


In [9]:
data = pd.read_parquet('df_reponse_daily_train.parquet')

In [14]:
data = data.reset_index()
data_filt = data[data['time'] >= '2015-01-01']
data_piv = data_filt.pivot(index='time', columns='stock_id')['21d_next_raw']

null_counts = data_piv.isnull().sum()
filtered_columns = null_counts[null_counts <= 10].index

data_processed_full = data_piv[filtered_columns]

In [53]:
random_columns = data_processed_full.sample(n=200, axis=1)
df_output_1 = random_columns.fillna(0)
df_output_1 = df_output_1.reset_index().reset_index(drop=True)

In [54]:
df_output_1.head()

stock_id,time,600369,600865,600838,2118,600883,2424,2495,600317,600470,...,554,2204,600895,2232,600613,600269,2644,686,600210,600476
0,2015-01-05 10:00:00,-0.167839,0.073394,0.164983,0.119966,0.043046,0.074813,-0.015231,-0.010460,0.100000,...,0.074371,0.013289,0.220190,0.129482,0.050207,-0.061144,0.070000,-0.178535,-0.023529,0.180749
1,2015-01-06 10:00:00,-0.152778,0.071151,0.141583,0.103194,0.057368,0.091544,-0.016724,-0.055336,0.065982,...,0.040749,0.028122,0.306938,0.136100,0.031250,-0.071705,0.062912,-0.164544,-0.011765,0.224055
2,2015-01-07 10:00:00,-0.157025,0.048656,0.161435,0.080658,0.110112,-0.004663,-0.024166,-0.032720,0.039474,...,0.021088,-0.015974,0.243811,0.119048,0.030040,-0.098077,0.056724,-0.164531,-0.031189,0.189260
3,2015-01-08 10:00:00,-0.118103,0.040868,0.124048,0.064491,0.062992,-0.043841,-0.071552,-0.069860,-0.012517,...,-0.018438,-0.014646,0.197434,0.121212,-0.008065,-0.140977,0.011478,-0.154497,-0.042718,0.162487
4,2015-01-09 10:00:00,-0.113362,0.068299,0.087719,0.028771,0.081515,-0.002565,-0.060947,-0.122841,-0.002813,...,0.016911,-0.088704,0.227306,0.087736,-0.012607,-0.130178,-0.011933,-0.147200,-0.052941,0.084126


In [90]:
df_output = pd.concat([df_output_1,factors], keys='time', axis=1).droplevel(0,axis=1)
df_output = df_output.loc[:, ~df_output.columns.duplicated()]
df_output.head()

,time,600369,600865,600838,2118,600883,2424,2495,600317,600470,...,fr_21d_next_raw_BETA,fr_21d_next_raw_RESVOL,fr_21d_next_raw_MOMENTUM,fr_21d_next_raw_LEVERAGE,fr_21d_next_raw_VALUE,fr_21d_next_raw_EARNYLD,fr_21d_next_raw_GROWTH,fr_21d_next_raw_LTREV,fr_21d_next_raw_STREV,fr_21d_next_raw_price
0,2015-01-05 10:00:00,-0.167839,0.073394,0.164983,0.119966,0.043046,0.074813,-0.015231,-0.010460,0.100000,...,0.003392,0.043820,-0.015595,-0.013227,-0.012587,0.011419,0.002194,-0.015348,0.033077,0.000531
1,2015-01-06 10:00:00,-0.152778,0.071151,0.141583,0.103194,0.057368,0.091544,-0.016724,-0.055336,0.065982,...,0.006909,0.054080,-0.015000,-0.007259,-0.014086,0.013329,0.002739,-0.017733,0.029639,0.000470
2,2015-01-07 10:00:00,-0.157025,0.048656,0.161435,0.080658,0.110112,-0.004663,-0.024166,-0.032720,0.039474,...,0.006934,0.054754,-0.017596,-0.002960,-0.010251,0.013611,0.004117,-0.016178,0.026539,0.000448
3,2015-01-08 10:00:00,-0.118103,0.040868,0.124048,0.064491,0.062992,-0.043841,-0.071552,-0.069860,-0.012517,...,0.009558,0.045922,-0.016463,-0.008067,-0.010321,0.016630,0.004974,-0.017005,0.030534,0.000425
4,2015-01-09 10:00:00,-0.113362,0.068299,0.087719,0.028771,0.081515,-0.002565,-0.060947,-0.122841,-0.002813,...,0.012897,0.034794,-0.012055,-0.009979,-0.011245,0.011686,0.006133,-0.017181,0.028227,0.000433


In [1]:
df_output.to_csv('train_returns_200rdn.csv', index=False)

NameError: name 'df_output' is not defined